# Data Preparation 

The input data for the network analysis part needs to be prepared in several steps. 

## Import Packages and set Directory

In [ ]:
import os
import pandas as pd
import numpy as np
#!pip install wikipedia-api
import wikipediaapi
import requests
from bs4 import BeautifulSoup

In [ ]:
# set directory
os.chdir('/content/drive/MyDrive/Social_Interactions/Data') # use your own directory!
os.getcwd()

In [ ]:
# get voting data (downloaded from https://www.parlament.ch/de/ratsbetrieb/abstimmungen/abstimmungs-datenbank-nr)
df50 = pd.read_csv('50th_legislature_2015_2019.csv', error_bad_lines=False)
df51 = pd.read_csv('51th_legislature.csv', error_bad_lines=False)

## Get the Party Affiliation of every Councillor 
They are not included in the files above

In [ ]:
# define language and page
wiki = wikipediaapi.Wikipedia('de')
page_name_party = wiki.page('Liste_der_Mitglieder_des_Schweizer_Nationalrats_in_der_50._Legislaturperiode')

# prepare beautiful soup parser
website_url = requests.get('https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Schweizer_Nationalrats_in_der_50._Legislaturperiode').text
soup = BeautifulSoup(website_url, 'html.parser')

# fetch data from wikipedia API
My_table = soup.findAll('table',{'class':'wikitable'})
My_table_person = My_table[0]
My_table_changes = My_table[1]

df_person = pd.read_html(str(My_table_person))
## convert list to dataframe
df_person =pd.DataFrame(df_person[0])

df_changes = pd.read_html(str(My_table_changes))
df_changes = pd.DataFrame(df_changes[0])
df_changes.head()

# save dataframes as csv files 
df_person.to_csv('Councillor_Party.csv')
df_changes.to_csv('Changes_during_period.csv')

## Define the Councillors which were re-elected and competed and actually competed in the Elections

In [ ]:
# create lists with councillor names
l_50 = list(set(df50['CouncillorName']))
l_51 = list(set(df51['CouncillorName']))

In [ ]:
# create two lists with councillor names, one for elected and one for not elected
elected = []
not_elected = []

for i in l_50:
  if i in l_51:
    elected.append(i)
  else:
    not_elected.append(i)

In [ ]:
# checked each person of the not_elected list, if wanted to be re-elected or not
not_competed = [0,1,2,3,6,7,8,9,10,11,12,16,17,18,23,24,25,26,
                   27,28,29,30,31,32,34,35,37,38,39,40,48,49,50,
                   51,53,54,56,57,58,61,62,63,64,65,66,69,70,71,
                   73,76,78,79,80,82,83,86,87,88,89,90,91,93]
not_reelected  = [4,5,13,14,15,19,20,21,22,33,36,41,42, 43,44,45,
                  46,47,52,55,59,60,67,68,72,74,75,77,81,84,85,92]

In [ ]:
not_elected_def =  []
for i in n_el:
  not_elected_def.append(not_elected[i])

In [ ]:
df_n_el = pd.DataFrame(not_elected_def, columns=['NotElectedCouncillors'])

In [ ]:
df_n_el.to_csv('not_elected_competed.csv')

## Get Dataframes from directory 

In [20]:
# get councillor and party data (I used the file from our google cloud folder)
df_party = pd.read_csv('Councillor_Party.csv', error_bad_lines=False)

In [21]:
# get df for not elected but competed councillors (I used the file from our google cloud folder)
df_not_elected = pd.read_csv('not_elected_competed.csv', error_bad_lines=False)

## Create lists with Partei Candidates

In [22]:
# create lists with partei councillors
SP = df_party.loc[df_party['Partei'] == 'SP']
SP['Name'] = SP.Name.str.split(" ").str[::-1].str.join(" ")

SVP = df_party.loc[df_party['Partei'] == 'SVP']
SVP['Name'] = SVP.Name.str.split(" ").str[::-1].str.join(" ")

CVP = df_party.loc[df_party['Partei'] == 'CVP']
CVP['Name'] = CVP.Name.str.split(" ").str[::-1].str.join(" ")

FDP = df_party.loc[df_party['Partei'] == 'FDP']
FDP['Name'] = FDP.Name.str.split(" ").str[::-1].str.join(" ")

GPS = df_party.loc[df_party['Partei'] == 'GPS']
GPS['Name'] = GPS.Name.str.split(" ").str[::-1].str.join(" ")

BDP = df_party.loc[df_party['Partei'] == 'BDP']
BDP['Name'] = BDP.Name.str.split(" ").str[::-1].str.join(" ")

glp = df_party.loc[df_party['Partei'] == 'glp']
glp['Name'] = glp.Name.str.split(" ").str[::-1].str.join(" ")

glp2 = df_party.loc[df_party['Partei'] == 'glp2']
glp2['Name'] = glp2.Name.str.split(" ").str[::-1].str.join(" ")

Lega = df_party.loc[df_party['Partei'] == 'Lega']
Lega['Name'] = Lega.Name.str.split(" ").str[::-1].str.join(" ")

EVP = df_party.loc[df_party['Partei'] == 'EVP']
EVP['Name'] = EVP.Name.str.split(" ").str[::-1].str.join(" ")

parteilos1 = df_party.loc[df_party['Partei'] == 'parteilos1']
parteilos1['Name'] = parteilos1.Name.str.split(" ").str[::-1].str.join(" ")

PdA = df_party.loc[df_party['Partei'] == 'PdA']
PdA['Name'] = PdA.Name.str.split(" ").str[::-1].str.join(" ")

<ipython-input-22-9e8341bd7701>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP['Name'] = SP.Name.str.split(" ").str[::-1].str.join(" ")


In [34]:
# add the partei as a column to the voting dataframe
df50['Partei'] = np.where(df50.CouncillorName.isin(SP.Name), 'SP',
                    np.where(df50.CouncillorName.isin(SVP.Name), 'SVP',
                    np.where(df50.CouncillorName.isin(CVP.Name), 'CVP',
                    np.where(df50.CouncillorName.isin(Lega.Name), 'Lega',
                    np.where(df50.CouncillorName.isin(parteilos1.Name), 'parteilos1',
                    np.where(df50.CouncillorName.isin(FDP.Name), 'FDP',
                    np.where(df50.CouncillorName.isin(GPS.Name), 'GPS',
                    np.where(df50.CouncillorName.isin(glp.Name), 'glp',
                    np.where(df50.CouncillorName.isin(glp2.Name), 'glp2',
                    np.where(df50.CouncillorName.isin(PdA.Name), 'PdA',
                    np.where(df50.CouncillorName.isin(BDP.Name), 'BDP',
                    np.where(df50.CouncillorName.isin(EVP.Name), 'EVP', '0'))))))))))))

In [35]:
# add a column with the reelected and not reelected but competed
df50['not_reelected'] = np.where(df50.CouncillorName.isin(df_not_elected.NotElectedCouncillors), '1', '0')
df50['reelected'] = np.where(df50.CouncillorName.isin(df51.CouncillorName), '1', '0')

0    755563
1    130905
Name: not_reelected, dtype: int64

## Create Party Dataframes

In [121]:
# create one dataframe for the right-winged fraction and one for the left-winged fraction
df_right_Fraktionen = df50.loc[df50['Partei'].isin(['SVP', 'Lega', 'parteilos1', 'FDP']) ]
df_left_Fraktionen = df50.loc[df50['Partei'].isin(['SP', 'GPS', 'glp', 'glp2', 'PdA', 'CVP', 'BDP', 'EVP']) ]

df_right_Fraktionen.to_csv('df_right_Fraktionen.csv')
df_left_Fraktionen.to_csv('df_left_Fraktionen.csv')

# create dataframes fpr Parties: SP, CVP, SVP
df_SP = df50.loc[df50['Partei'] == 'SP']
df_CVP = df50.loc[df50['Partei'] == 'CVP']
df_SVP = df50.loc[df50['Partei'] == 'SVP']

df_SP.to_csv('df_SP.csv')
df_CVP.to_csv('df_CVP.csv')
df_SVP.to_csv('df_SVP.csv')

,AffairShortId,AffairTitle,VoteRegistrationNumber,VoteDate,VoteMeaningYes,VoteMeaningNo,DivisionText,VoteSubmissionText,VoteFilteredYes,VoteFilteredNo,...,CouncillorName,CouncillorYes,CouncillorNo,CouncillorAbstain,CouncillorNotParticipated,CouncillorExcused,CouncillorPresident,Partei,not_reelected,reelected
2,20193033,Offensive zur Stärkung des Wirtschaftsstandort...,19593,Thu Sep 26 2019 20:05:17 GMT+0200 (Mitteleurop...,Accettare la mozione,Respingere la mozione,Ja,NaN,121,46,...,Glauser-Zufferey Alice,1,0,0,0,0,0,SVP,1,0
5,20193033,Offensive zur Stärkung des Wirtschaftsstandort...,19593,Thu Sep 26 2019 20:05:17 GMT+0200 (Mitteleurop...,Accettare la mozione,Respingere la mozione,Ja,NaN,121,46,...,Estermann Yvette,1,0,0,0,0,0,SVP,0,1
7,20193033,Offensive zur Stärkung des Wirtschaftsstandort...,19593,Thu Sep 26 2019 20:05:17 GMT+0200 (Mitteleurop...,Accettare la mozione,Respingere la mozione,Ja,NaN,121,46,...,Flückiger-Bäni Sylvia,1,0,0,0,0,0,SVP,0,0
25,20193033,Offensive zur Stärkung des Wirtschaftsstandort...,19593,Thu Sep 26 2019 20:05:17 GMT+0200 (Mitteleurop...,Accettare la mozione,Respingere la mozione,Ja,NaN,121,46,...,Nidegger Yves,1,0,0,0,0,0,SVP,0,1
27,20193033,Offensive zur Stärkung des Wirtschaftsstandort...,19593,Thu Sep 26 2019 20:05:17 GMT+0200 (Mitteleurop...,Accettare la mozione,Respingere la mozione,Hat nicht teilgenommen,NaN,121,46,...,Reimann Lukas,0,0,0,1,0,0,SVP,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886373,20070428,Strafrechtsrevision rückgängig machen bezüglic...,12622,Tue Dec 01 2015 13:45:59 GMT+0100 (Mitteleurop...,Antrag der Mehrheit (keine Folge geben),Antrag der Minderheit Stamm (Folge geben),Nein,NaN,123,67,...,Müller Thomas,0,1,0,0,0,0,SVP,0,0
886374,20070428,Strafrechtsrevision rückgängig machen bezüglic...,12622,Tue Dec 01 2015 13:45:59 GMT+0100 (Mitteleurop...,Antrag der Mehrheit (keine Folge geben),Antrag der Minderheit Stamm (Folge geben),Nein,NaN,123,67,...,Dettling Marcel,0,1,0,0,0,0,SVP,0,1
886375,20070428,Strafrechtsrevision rückgängig machen bezüglic...,12622,Tue Dec 01 2015 13:45:59 GMT+0100 (Mitteleurop...,Antrag der Mehrheit (keine Folge geben),Antrag der Minderheit Stamm (Folge geben),Nein,NaN,123,67,...,Grüter Franz,0,1,0,0,0,0,SVP,0,1
886376,20070428,Strafrechtsrevision rückgängig machen bezüglic...,12622,Tue Dec 01 2015 13:45:59 GMT+0100 (Mitteleurop...,Antrag der Mehrheit (keine Folge geben),Antrag der Minderheit Stamm (Folge geben),Nein,NaN,123,67,...,Reimann Maximilian,0,1,0,0,0,0,SVP,1,0
